<a href="https://colab.research.google.com/github/svenklingel/airbnb-price-prediction/blob/main/AirBnB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osmnx mapclassify folium matplotlib catboost optuna dcor
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import numpy as np
import optuna
import osmnx as ox
import dcor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from math import sqrt
from sklearn.metrics import root_mean_squared_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from shapely.geometry import Point
from scipy.stats import spearmanr
from shapely.geometry import box
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, RandomizedSearchCV

Data exploration


In [ ]:
airbnb_df = pd.read_csv(r"/content/AB_NYC_2019.csv")
airbnb_df.head()

In [ ]:
# GeoDataFrame created from Dataframe and latitude/longitude columns
airbnb_gdf = gpd.GeoDataFrame(airbnb_df, geometry=gpd.points_from_xy(airbnb_df.longitude, airbnb_df.latitude))
airbnb_gdf.set_crs('epsg:4326', inplace=True)
airbnb_gdf.head()

In [ ]:
# 1. Load boroughs polygons
boroughs_gdf = gpd.read_file("boroughs.json")

# 2. Ensure the same CRS for both GeoDataFrames
airbnb_gdf = airbnb_gdf.to_crs(epsg=2263)
boroughs_gdf = boroughs_gdf.to_crs(epsg=2263)

# 3. Spatial join: assign points to borough polygons
joined_gdf = gpd.sjoin(airbnb_gdf, boroughs_gdf, how="inner", predicate="within")

# 4. Count listings per borough
listing_counts = joined_gdf['boro_name'].value_counts().rename_axis('boro_name').reset_index(name='listing_count')

# 5. Calculate area (in km²)
boroughs_gdf['area_km2'] = boroughs_gdf.geometry.area / 1e6

# 6. Merge boroughs with listing counts
boroughs_gdf = boroughs_gdf.merge(listing_counts, on='boro_name', how='left')
boroughs_gdf['listing_count'] = boroughs_gdf['listing_count'].fillna(0)

# 7. Calculate density (Listings/km²)
boroughs_gdf['density'] = boroughs_gdf['listing_count'] / boroughs_gdf['area_km2']

# 8. Calculate proportional share (in %)
total_listings = boroughs_gdf['listing_count'].sum()
boroughs_gdf['proportion'] = (boroughs_gdf['listing_count'] / total_listings) * 100

# 9. Label with all information
boroughs_gdf['label'] = boroughs_gdf.apply(
    lambda row: f"{row['boro_name']} ({int(row['listing_count'])} Listings, "
                f"{row['density']:.1f}/km², {row['proportion']:.1f}%)", axis=1
)

boroughs_map = boroughs_gdf.explore(
    column="label",
    cmap="Set1",
    tooltip=["boro_name", "proportion"],
    legend=True,
    legend_kwds={
        "caption": "Boroughs and proportion of listings",
        "colorbar": False
    }
)

listings_map = airbnb_gdf.explore(m=boroughs_map)
listings_map


Exploration of column meanings, datatypes, missing values, distributions, skewness etc.

In [ ]:
# Duplicated rows
airbnb_gdf.duplicated().sum() # 0 Duplicates

In [ ]:
# Missing values
airbnb_gdf.isna().sum()

In [ ]:
# Unique values per column
airbnb_gdf.nunique()

In [ ]:
# Unique neighbourhood groups
airbnb_gdf['neighbourhood_group'].unique()


In [ ]:
# Proportion of listenings per borough
borough_counts = airbnb_gdf['neighbourhood_group'].value_counts()
borough_counts.plot.pie(autopct='%1.1f%%')
plt.title('Listenings per Borough',loc="center")
plt.ylabel('')
plt.show()

In [ ]:
# group by neighbourhood_group and calculate median price
borough_price = airbnb_gdf.groupby('neighbourhood_group')['price'].median()

borough_price = borough_price.to_dict()

labels = list(borough_price.keys())
values = list(borough_price.values())

plt.figure(figsize=(8, 5))
bars = plt.bar(labels, values, color='cornflowerblue')
plt.ylabel("Median price ($)")
plt.title("Median price per borough")
plt.xticks(rotation=20)

# Werte über die Balken schreiben
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height , f"{height:.2f} $", ha='center', va='bottom')

plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()


In [ ]:
# Unique room types
airbnb_gdf['room_type'].unique()

In [ ]:
# Proportion, counts of room types
room_type_counts = airbnb_gdf['room_type'].value_counts()
room_type_counts


In [ ]:
room_type_counts.plot.pie(autopct='%1.1f%%')
plt.title('Listings per room type',loc="center")
plt.ylabel('')
plt.show()

In [ ]:
# group by neighbourhood_group and calculate average price
room_type_price = airbnb_gdf.groupby('room_type')['price'].median()
room_type_price = room_type_price.to_dict()

labels = list(room_type_price.keys())
values = list(room_type_price.values())

plt.figure(figsize=(8, 5))
bars = plt.bar(labels, values, color='cornflowerblue')
plt.ylabel("Median price ($)")
plt.title("Median price per room type")
plt.xticks(rotation=20)

# Werte über die Balken schreiben
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height , f"{height:.2f} $", ha='center', va='bottom')

plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()


In [ ]:
# Spatial distribution per room type
roomtype_map = airbnb_gdf.explore(
    column="room_type",
    cmap="tab10",
    legend=True,
    legend_kwds={
        "caption": "Distribution of room type",
        "colorbar": False
    }
)
roomtype_map

In [ ]:
# Proportion of listenings per neighbourhood
neighbourhood_percentage = airbnb_gdf['neighbourhood'].value_counts(normalize=True).mul(100).round(1).astype(str)
neighbourhood_percentage.sort_values(ascending=True)

In [ ]:
# Number of unique neighbourhoods
len(airbnb_gdf['neighbourhood'].unique())

In [ ]:
# Histogram before cleaning
# Numeric columns
numeric_cols = airbnb_gdf.select_dtypes(include='number').columns.tolist()
numeric_cols.append("calculated_host_listings_count")
# Not relevant columns
for col in ["id", "host_id", "longitude", "latitude"]:
    if col in numeric_cols:
        numeric_cols.remove(col)

# Histograms
for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    plt.hist(airbnb_df[col].dropna(), bins=200, edgecolor='k', color='skyblue')
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()


In [ ]:
# Scatterplots against 'price' before cleaning
for col in numeric_cols:

    plt.figure(figsize=(6, 4))
    plt.scatter(airbnb_df[col], airbnb_df['price'], alpha=0.5, edgecolor='k')
    plt.title(f'Price vs {col}')
    plt.xlabel(col)
    plt.ylabel('Price')
    plt.tight_layout()
    plt.show()

In [ ]:
# Check correlations before data cleaning
cols_for_correlation = ['price', 'minimum_nights', 'number_of_reviews',
                        'reviews_per_month', 'calculated_host_listings_count',
                        'availability_365', 'longitude', 'latitude']

# Pearson correlation (linear)
pearson_corr_orig = airbnb_gdf[cols_for_correlation].corr(method='pearson')

plt.figure(figsize=(10, 8))
sns.heatmap(pearson_corr_orig, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Pearson Correlation")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Spearman correlation (monotonic, non-linear)
spearman_corr = airbnb_gdf[cols_for_correlation].corr(method='spearman')

plt.figure(figsize=(10,8))
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Spearman Correlation")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


Data cleaning

In [ ]:
# Metrics of the target ariable
airbnb_df['price'].describe()

In [ ]:
# Boxplot of price
airbnb_gdf.boxplot(column='price')

In [ ]:
# Price outliers: Remove the first and last 1% outliers (below 1th percentile, above 95th percentile)
lower_bound = airbnb_gdf['price'].quantile(0.01)
upper_bound = airbnb_gdf['price'].quantile(0.95)

print(f"Lower Bound (1th percentile): {lower_bound}")
print(f"Upper Bound (95th percentile): {upper_bound}")

# Count of outliers below the lower bound
outliers_below = airbnb_gdf[airbnb_gdf['price'] < lower_bound].shape[0]
print(f"Number of outliers below the lower bound: {outliers_below}")

# Count of outliers above the upper bound
outliers_above = airbnb_gdf[airbnb_gdf['price'] > upper_bound].shape[0]
print(f"Number of outliers above the upper bound: {outliers_above}")

# Filter the data to remove outliers
filtered_gdf = airbnb_gdf[(airbnb_gdf['price'] >= lower_bound) & (airbnb_gdf['price'] <= upper_bound)]

# Shapes of old and filtered data
print(f"Original DataFrame shape: {airbnb_gdf.shape}")
print(f"Filtered DataFrame shape: {filtered_gdf.shape}")


In [ ]:
# Histogram of price of filtered_df
plt.hist(filtered_gdf['price'], bins=250, edgecolor='k')
plt.title('Price without outliers')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Logarithmic transformation of price to reuce right skew
filtered_gdf['price_log'] = np.log(filtered_gdf['price'])
plt.hist(filtered_gdf['price_log'], bins=200, edgecolor='k')
plt.title('Filtered and log. transformed price')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Numeric columns after cleaning based on price variable
numeric_cols = filtered_gdf.select_dtypes(include='number').columns.tolist()

for col in ["id", "host_id", "longitude", "latitude"]:
    if col in numeric_cols:
        numeric_cols.remove(col)

numeric_cols

for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    plt.hist(filtered_gdf[col].dropna(), bins=200, edgecolor='k', color='skyblue')
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()


In [ ]:
# Scatterplots against 'price' after cleaning
for col in numeric_cols:

    plt.figure(figsize=(6, 4))
    plt.scatter(airbnb_df[col], airbnb_df['price'], alpha=0.5, edgecolor='k')
    plt.title(f'Price vs {col}')
    plt.xlabel(col)
    plt.ylabel('Price')
    plt.tight_layout()
    plt.show()


In [ ]:
# Scatterplots against 'price' after applying log transformation
for col in numeric_cols:
    plt.figure(figsize=(6, 4))

    # Apply log transformation to both axes
    plt.scatter(np.log1p(airbnb_df[col]), np.log1p(airbnb_df['price']), alpha=0.5, edgecolor='k')

    # Set the title and labels
    plt.title(f'Log(Price) vs Log({col})')
    plt.xlabel(f'Log({col})')
    plt.ylabel('Log(Price)')

    plt.tight_layout()
    plt.show()


Feature engineering

In [ ]:
cols_for_correlation = ['price', 'minimum_nights', 'number_of_reviews',
                        'reviews_per_month', 'calculated_host_listings_count',
                        'availability_365', 'longitude', 'latitude']

In [ ]:
# Correaltions after cleaning
# Pearson correlation better for linear relationships
pearson_corr = filtered_gdf[cols_for_correlation].corr(method='pearson')

# Correlation hetammap
plt.figure(figsize=(10, 8))
sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Pearson correlation")

plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()
plt.show()



# Spearman correlation better for non-linear relationships
spearman_corr = filtered_gdf[cols_for_correlation].corr(method='spearman')

plt.figure(figsize=(10,8))
ax = sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

plt.title("Spearman correlation")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Relevance of several distance based features
# Merged multipolygon of all boroughs
multi_poly = boroughs_gdf.dissolve()
multi_poly.explore()

In [ ]:
# MutiPolygon of all boroughs used to fetch OSM data
multi_poly_4326 = multi_poly.to_crs(epsg=4326)
poly=multi_poly_4326["geometry"][0]

# Restaurants inside the MultiPoly
tags = {"amenity": "restaurant"}
restaurants_gdf = ox.features_from_polygon(poly, tags)

# Reprojection to EPSG:3857 fpr distance calculation
filtered_gdf.to_crs(epsg=3857, inplace=True)
restaurants_gdf.to_crs(epsg=3857, inplace=True)

restaurants_gdf.explore()


In [ ]:
print(len(restaurants_gdf))

In [ ]:
# Function for calculation of the shortest disatnce between an Airbnb offer to a restaurant
def nearest_distance(point, other_gdf):
    distances = other_gdf.geometry.distance(point)
    return distances.min()

# Distance calculation for restaurants
filtered_gdf['dist_restaurant'] = filtered_gdf.geometry.apply(lambda x: nearest_distance(x, restaurants_gdf))

# Correlation between dist_restaurant and price
corr_restau, p_restau = spearmanr(filtered_gdf['dist_restaurant'], filtered_gdf['price'])
print(f"Spearman-Correlation: {corr_restau}, p-Value: {p_restau}")

In [ ]:
# Railways
tags = {"railway": "station",
        "railway": "subway_entrance"}

railway_gdf = ox.features_from_polygon(poly, tags)
railway_gdf.to_crs(epsg=3857, inplace=True)
filtered_gdf.explore()


In [ ]:
print(len(railway_gdf))

In [ ]:
filtered_gdf['dist_railway'] = filtered_gdf.geometry.apply(lambda x: nearest_distance(x, railway_gdf))

corr_railway, p_railway = spearmanr(filtered_gdf['dist_railway'], filtered_gdf['price'])
print(f"Spearman-Correlation: {corr_railway}, p-Value: {p_railway}")

In [ ]:
# Attractions
tags = {"tourism": "attraction",}
attraction_gdf = ox.features_from_polygon(poly, tags)
attraction_gdf.to_crs(epsg=3857, inplace=True)
attraction_gdf.explore()

In [ ]:
print(len(attraction_gdf))

In [ ]:
filtered_gdf['dist_attraction'] = filtered_gdf.geometry.apply(lambda x: nearest_distance(x, attraction_gdf))

corr_attraction, p_attraction = spearmanr(filtered_gdf['dist_attraction'], filtered_gdf['price'])
print(f"Spearman-Correlation: {corr_attraction}, p-Value: {p_attraction}")

In [ ]:
# Coastline
tags = {"natural": "coastline"}

# Bounding Box (minx, miny, maxx, maxy)
bounds = multi_poly_4326.total_bounds

# Shapely polygon
poly = box(*bounds)

coastline_gdf = ox.features_from_polygon(poly, tags)

coastline_gdf.to_crs(epsg=3857, inplace=True)
coastline_gdf = coastline_gdf[(coastline_gdf.geometry.type == 'LineString') | (coastline_gdf.geometry.type == 'MultiLineString')]
coastline_gdf.explore()

In [ ]:
# Distance to coastline
coastline_geom = coastline_gdf.geometry.union_all()
filtered_gdf["dist_coastline"] = filtered_gdf.geometry.distance(coastline_geom)
filtered_gdf["dist_coastline"]

# Correlation between dist_restaurant and price
corr_coast, p_coast = spearmanr(filtered_gdf['dist_coastline'], filtered_gdf['price'])
print(f"Spearman-Correlation: {corr_coast}, p-Value: {p_coast}")

In [ ]:
# Distance based  features
dist_cols = ['dist_railway', 'dist_attraction', 'dist_restaurant', "dist_coastline"]

# Calculation of correlations with price
correlations = {}
for col in dist_cols:
    corr, _ = spearmanr(filtered_gdf[col], filtered_gdf['price'])
    correlations[col] = corr

corr_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Spearman correlation'])

# Heatmap
plt.figure(figsize=(6, 3))
sns.heatmap(corr_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f", linewidths=0.5)
plt.title('Distance variables vs. price')
plt.ylabel('Distance variable')
plt.xlabel('')
plt.tight_layout()
plt.show()


Suitable features:
- room_type
- neighborhood_group,
- neighborhood
- dist_attraction
- dist_coastline
- dist_restaurant

In [ ]:
# Selection of features
features_df = filtered_gdf[['price_log','neighbourhood_group', 'room_type','neighbourhood','calculated_host_listings_count', 'minimum_nights', 'availability_365', 'dist_attraction', 'dist_coastline', 'dist_restaurant', 'reviews_per_month', 'number_of_reviews']]
features_df.head()

In [ ]:
# One-hot-encoding for cat vars
features_encoded = pd.get_dummies(features_df, columns=['room_type', 'neighbourhood_group','neighbourhood'], drop_first=True)
#features_encoded = features_df

Model training and evaluation

In [ ]:
# Load and prepare data
X = features_encoded.drop("price_log", axis=1)
y = features_encoded["price_log"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------- Optuna Tuning for XGBoost --------------------
def objective_xgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 5, 500),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0)
    }
    model = XGBRegressor(**params, random_state=42, verbosity=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = root_mean_squared_error(y_test, preds)
    return rmse

study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_xgb, n_trials=25)
print("Best XGBoost params:", study_xgb.best_params)
print("Best XGBoost RMSE", study_xgb.best_value)

# -------------------- Optuna Tuning for CatBoost --------------------
def objective_cat(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 5, 500),
        "depth": trial.suggest_int("depth", 2, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
    }
    model = CatBoostRegressor(**params, verbose=0, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = root_mean_squared_error(y_test, preds)
    return rmse

study_cat = optuna.create_study(direction="minimize")
study_cat.optimize(objective_cat, n_trials=25)
print("Best CatBoost params:", study_cat.best_params)
print("CatBoost RMSE:", study_cat.best_value)

In [ ]:
# Final XGBoost model
xgboost_model = XGBRegressor(**study_xgb.best_params, random_state=42, verbosity=0)
xgboost_model.fit(X_train, y_train)

xgboost_preds_log = xgboost_model.predict(X_test)
xgboost_rmse_log = root_mean_squared_error(y_test, xgboost_preds_log)

xgboost_preds_dollar = np.exp(xgboost_preds_log)
y_test_dollar = np.exp(y_test)
xgboost_rmse_dollar = root_mean_squared_error(y_test_dollar, xgboost_preds_dollar)

print(f"XGBoost RMSE (log scale): {xgboost_rmse_log}")
print(f"XGBoost RMSE (dollar): {xgboost_rmse_dollar}")
# Feature Importances (Gini importance / Gain)
importances = xgboost_model.feature_importances_
feature_names = X_train.columns

importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Ausgabe in Konsole
print("\nCatBoost Feature Importances:")
print(importance_df.to_string(index=False))

CatBoost RMSE (log scale): 0.33777854302676613

In [ ]:
# Final Catboost model
catboost_model = CatBoostRegressor(**study_cat.best_params, verbose=0, random_state=42)
catboost_model.fit(X_train, y_train)

catboost_preds_log = catboost_model.predict(X_test)
catboost_rmse_log = root_mean_squared_error(y_test, catboost_preds_log)

catboost_preds_dollar = np.exp(catboost_preds_log)
y_test_dollar = np.exp(y_test)
catboost_rmse_dollar = root_mean_squared_error(y_test_dollar, catboost_preds_dollar)

print(f"CatBoost RMSE (log scale): {catboost_rmse_log}")
print(f"CatBoost RMSE (dollar): {catboost_rmse_dollar}")
# Feature Importances
importances = catboost_model.get_feature_importance()
feature_names = X_train.columns

importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Ausgabe in Konsole
print("\nCatBoost Feature Importances:")
print(importance_df.to_string(index=False))